In [ ]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import losses
from keras.utils import to_categorical
import keras.backend as K

import matplotlib.pyplot as plt

import numpy as np

class SGAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy', 'categorical_crossentropy'],
            loss_weights=[0.5, 0.5],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        noise = Input(shape=(100,))
        img = self.generator(noise)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines valid
        valid, _ = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model(noise , valid)
        self.combined.compile(loss=['binary_crossentropy'],
            optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(1, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())

        model.summary()

        img = Input(shape=self.img_shape)

        features = model(img)
        valid = Dense(1, activation="sigmoid")(features)
        label = Dense(self.num_classes+1, activation="softmax")(features)

        return Model(img, [valid, label])

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Class weights:
        # To balance the difference in occurences of digit class labels.
        # 50% of labels that the discriminator trains on are 'fake'.
        # Weight = 1 / frequency
        cw1 = {0: 1, 1: 1}
        cw2 = {i: self.num_classes / 64 for i in range(self.num_classes)}
        cw2[self.num_classes] = 1 / 64

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # One-hot encoding of labels
            labels = to_categorical(y_train[idx], num_classes=self.num_classes+1)
            fake_labels = to_categorical(np.full((batch_size, 1), self.num_classes), num_classes=self.num_classes+1)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, [valid, labels], class_weight=[cw1, cw2])
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, [fake, fake_labels], class_weight=[cw1, cw2])
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            # ---------------------
            #  Train Generator
            # ---------------------

            g_loss = self.combined.train_on_batch(noise, valid, class_weight=[cw1, cw2])

            # Plot the progress
            print ("%d [D loss: %f, acc: %.2f%%, op_acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[3], 100*d_loss[4], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 1

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/mnist_%d.png" % epoch)
        plt.close()

    def save_model(self):

        def save(model, model_name):
            model_path = "saved_model/%s.json" % model_name
            weights_path = "saved_model/%s_weights.hdf5" % model_name
            options = {"file_arch": model_path,
                        "file_weight": weights_path}
            json_string = model.to_json()
            open(options['file_arch'], 'w').write(json_string)
            model.save_weights(options['file_weight'])

        save(self.generator, "mnist_sgan_generator")
        save(self.discriminator, "mnist_sgan_discriminator")
        save(self.combined, "mnist_sgan_adversarial")


if __name__ == '__main__':
    sgan = SGAN()
    sgan.train(epochs=20000, batch_size=32, sample_interval=50)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 8, 8, 64)          0         
__________

C:\Users\adward\Anaconda3\lib\site-packages\keras\engine\training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.579261, acc: 39.06%, op_acc: 7.81%] [G loss: 0.713937]
1 [D loss: 0.416571, acc: 70.31%, op_acc: 7.81%] [G loss: 0.773397]
2 [D loss: 0.428133, acc: 64.06%, op_acc: 7.81%] [G loss: 0.821273]
3 [D loss: 0.366195, acc: 75.00%, op_acc: 17.19%] [G loss: 1.253886]
4 [D loss: 0.316966, acc: 81.25%, op_acc: 14.06%] [G loss: 1.117550]
5 [D loss: 0.299595, acc: 89.06%, op_acc: 12.50%] [G loss: 1.096185]
6 [D loss: 0.261588, acc: 87.50%, op_acc: 15.62%] [G loss: 1.531133]
7 [D loss: 0.255547, acc: 92.19%, op_acc: 17.19%] [G loss: 1.341451]
8 [D loss: 0.241427, acc: 93.75%, op_acc: 7.81%] [G loss: 1.822409]
9 [D loss: 0.251653, acc: 90.62%, op_acc: 7.81%] [G loss: 1.598400]
10 [D loss: 0.260286, acc: 87.50%, op_acc: 9.38%] [G loss: 1.976312]
11 [D loss: 0.247217, acc: 92.19%, op_acc: 18.75%] [G loss: 2.247524]
12 [D loss: 0.240856, acc: 92.19%, op_acc: 20.31%] [G loss: 2.127833]
13 [D loss: 0.223165, acc: 98.44%, op_acc: 20.31%] [G loss: 2.329797]
14 [D loss: 0.200274, acc: 96.88%, o

117 [D loss: 0.372315, acc: 60.94%, op_acc: 35.94%] [G loss: 1.789702]
118 [D loss: 0.555577, acc: 53.12%, op_acc: 39.06%] [G loss: 1.851548]
119 [D loss: 0.416960, acc: 51.56%, op_acc: 46.88%] [G loss: 2.325421]
120 [D loss: 0.458810, acc: 56.25%, op_acc: 39.06%] [G loss: 1.901093]
121 [D loss: 0.372144, acc: 56.25%, op_acc: 42.19%] [G loss: 1.656196]
122 [D loss: 0.353309, acc: 73.44%, op_acc: 35.94%] [G loss: 2.038827]
123 [D loss: 0.365607, acc: 65.62%, op_acc: 43.75%] [G loss: 2.030883]
124 [D loss: 0.550374, acc: 54.69%, op_acc: 42.19%] [G loss: 1.397941]
125 [D loss: 0.423733, acc: 54.69%, op_acc: 35.94%] [G loss: 1.631450]
126 [D loss: 0.405314, acc: 57.81%, op_acc: 46.88%] [G loss: 1.935527]
127 [D loss: 0.498778, acc: 54.69%, op_acc: 42.19%] [G loss: 1.894902]
128 [D loss: 0.435021, acc: 54.69%, op_acc: 39.06%] [G loss: 1.475086]
129 [D loss: 0.321941, acc: 75.00%, op_acc: 50.00%] [G loss: 1.981306]
130 [D loss: 0.441993, acc: 65.62%, op_acc: 43.75%] [G loss: 1.595490]
131 [D

233 [D loss: 0.411657, acc: 53.12%, op_acc: 45.31%] [G loss: 1.269415]
234 [D loss: 0.380746, acc: 60.94%, op_acc: 43.75%] [G loss: 1.637947]
235 [D loss: 0.469757, acc: 53.12%, op_acc: 45.31%] [G loss: 1.532785]
236 [D loss: 0.390993, acc: 59.38%, op_acc: 50.00%] [G loss: 1.605288]
237 [D loss: 0.400537, acc: 53.12%, op_acc: 53.12%] [G loss: 1.291808]
238 [D loss: 0.323269, acc: 71.88%, op_acc: 50.00%] [G loss: 1.478131]
239 [D loss: 0.354394, acc: 65.62%, op_acc: 56.25%] [G loss: 1.404700]
240 [D loss: 0.389236, acc: 68.75%, op_acc: 46.88%] [G loss: 1.182510]
241 [D loss: 0.367607, acc: 62.50%, op_acc: 45.31%] [G loss: 1.339871]
242 [D loss: 0.424097, acc: 60.94%, op_acc: 48.44%] [G loss: 1.298168]
243 [D loss: 0.423205, acc: 48.44%, op_acc: 54.69%] [G loss: 1.446918]
244 [D loss: 0.336517, acc: 71.88%, op_acc: 48.44%] [G loss: 1.656642]
245 [D loss: 0.462055, acc: 48.44%, op_acc: 45.31%] [G loss: 1.383900]
246 [D loss: 0.403031, acc: 59.38%, op_acc: 46.88%] [G loss: 1.406032]
247 [D

...

16152 [D loss: 0.303533, acc: 68.75%, op_acc: 62.50%] [G loss: 1.559100]
16153 [D loss: 0.245566, acc: 82.81%, op_acc: 56.25%] [G loss: 1.519019]
16154 [D loss: 0.395629, acc: 54.69%, op_acc: 59.38%] [G loss: 1.026447]
16155 [D loss: 0.386413, acc: 59.38%, op_acc: 56.25%] [G loss: 0.762844]
16156 [D loss: 0.311601, acc: 65.62%, op_acc: 56.25%] [G loss: 1.067555]
16157 [D loss: 0.249943, acc: 76.56%, op_acc: 57.81%] [G loss: 1.223190]
16158 [D loss: 0.305255, acc: 67.19%, op_acc: 64.06%] [G loss: 1.417946]
16159 [D loss: 0.381798, acc: 57.81%, op_acc: 57.81%] [G loss: 1.026335]
16160 [D loss: 0.261286, acc: 79.69%, op_acc: 59.38%] [G loss: 1.273762]
16161 [D loss: 0.256657, acc: 76.56%, op_acc: 54.69%] [G loss: 1.157647]
16162 [D loss: 0.197912, acc: 87.50%, op_acc: 65.62%] [G loss: 1.150183]
16163 [D loss: 0.334522, acc: 62.50%, op_acc: 54.69%] [G loss: 1.572555]
16164 [D loss: 0.276924, acc: 73.44%, op_acc: 54.69%] [G loss: 1.745076]
16165 [D loss: 0.392221, acc: 56.25%, op_acc: 53.12

16265 [D loss: 0.287344, acc: 68.75%, op_acc: 54.69%] [G loss: 1.549513]
16266 [D loss: 0.269897, acc: 76.56%, op_acc: 64.06%] [G loss: 1.304976]
16267 [D loss: 0.454981, acc: 37.50%, op_acc: 50.00%] [G loss: 1.217463]
16268 [D loss: 0.286055, acc: 75.00%, op_acc: 51.56%] [G loss: 0.705788]
16269 [D loss: 0.306483, acc: 68.75%, op_acc: 60.94%] [G loss: 1.116116]
16270 [D loss: 0.386155, acc: 57.81%, op_acc: 64.06%] [G loss: 0.822788]
16271 [D loss: 0.382494, acc: 56.25%, op_acc: 53.12%] [G loss: 1.052862]
16272 [D loss: 0.277731, acc: 75.00%, op_acc: 57.81%] [G loss: 1.121887]
16273 [D loss: 0.281160, acc: 70.31%, op_acc: 65.62%] [G loss: 1.613649]
16274 [D loss: 0.339824, acc: 70.31%, op_acc: 56.25%] [G loss: 1.405222]
16275 [D loss: 0.479123, acc: 42.19%, op_acc: 48.44%] [G loss: 1.362185]
16276 [D loss: 0.319680, acc: 65.62%, op_acc: 54.69%] [G loss: 1.466500]
16277 [D loss: 0.479052, acc: 37.50%, op_acc: 45.31%] [G loss: 0.869005]
16278 [D loss: 0.235705, acc: 81.25%, op_acc: 56.25

16378 [D loss: 0.350973, acc: 57.81%, op_acc: 51.56%] [G loss: 1.201316]
16379 [D loss: 0.257321, acc: 78.12%, op_acc: 60.94%] [G loss: 2.147707]
16380 [D loss: 0.241478, acc: 79.69%, op_acc: 67.19%] [G loss: 1.669654]
16381 [D loss: 0.335017, acc: 60.94%, op_acc: 53.12%] [G loss: 1.173324]
16382 [D loss: 0.394597, acc: 51.56%, op_acc: 57.81%] [G loss: 1.457124]
16383 [D loss: 0.290276, acc: 75.00%, op_acc: 56.25%] [G loss: 1.510238]
16384 [D loss: 0.342778, acc: 59.38%, op_acc: 56.25%] [G loss: 1.030551]
16385 [D loss: 0.372348, acc: 64.06%, op_acc: 54.69%] [G loss: 0.949948]
16386 [D loss: 0.386763, acc: 57.81%, op_acc: 46.88%] [G loss: 1.137791]
16387 [D loss: 0.228582, acc: 81.25%, op_acc: 67.19%] [G loss: 1.604697]
16388 [D loss: 0.333052, acc: 65.62%, op_acc: 53.12%] [G loss: 0.881140]
16389 [D loss: 0.222032, acc: 84.38%, op_acc: 62.50%] [G loss: 1.771398]
16390 [D loss: 0.345467, acc: 67.19%, op_acc: 54.69%] [G loss: 1.022553]
16391 [D loss: 0.353361, acc: 60.94%, op_acc: 50.00

16491 [D loss: 0.359216, acc: 51.56%, op_acc: 50.00%] [G loss: 1.108444]
16492 [D loss: 0.291191, acc: 73.44%, op_acc: 51.56%] [G loss: 1.475507]
16493 [D loss: 0.248871, acc: 78.12%, op_acc: 65.62%] [G loss: 1.476244]
16494 [D loss: 0.311802, acc: 70.31%, op_acc: 59.38%] [G loss: 1.201777]
16495 [D loss: 0.285880, acc: 67.19%, op_acc: 56.25%] [G loss: 1.720748]
16496 [D loss: 0.271443, acc: 76.56%, op_acc: 67.19%] [G loss: 1.629208]
16497 [D loss: 0.348565, acc: 62.50%, op_acc: 68.75%] [G loss: 1.857930]
16498 [D loss: 0.313737, acc: 67.19%, op_acc: 46.88%] [G loss: 1.151475]
16499 [D loss: 0.260514, acc: 76.56%, op_acc: 59.38%] [G loss: 1.559296]
16500 [D loss: 0.293684, acc: 76.56%, op_acc: 62.50%] [G loss: 1.743276]
16501 [D loss: 0.408483, acc: 42.19%, op_acc: 60.94%] [G loss: 0.886130]
16502 [D loss: 0.241870, acc: 78.12%, op_acc: 64.06%] [G loss: 1.276763]
16503 [D loss: 0.233587, acc: 81.25%, op_acc: 59.38%] [G loss: 1.431647]
16504 [D loss: 0.368124, acc: 59.38%, op_acc: 57.81

16604 [D loss: 0.422214, acc: 51.56%, op_acc: 51.56%] [G loss: 0.994362]
16605 [D loss: 0.311734, acc: 70.31%, op_acc: 53.12%] [G loss: 0.935495]
16606 [D loss: 0.353511, acc: 54.69%, op_acc: 54.69%] [G loss: 1.010638]
16607 [D loss: 0.518974, acc: 45.31%, op_acc: 46.88%] [G loss: 1.229740]
16608 [D loss: 0.354427, acc: 56.25%, op_acc: 54.69%] [G loss: 1.289846]
16609 [D loss: 0.318622, acc: 70.31%, op_acc: 64.06%] [G loss: 1.084118]
16610 [D loss: 0.343554, acc: 64.06%, op_acc: 57.81%] [G loss: 1.003304]
16611 [D loss: 0.352648, acc: 65.62%, op_acc: 54.69%] [G loss: 1.387765]
16612 [D loss: 0.327818, acc: 67.19%, op_acc: 59.38%] [G loss: 1.326305]
16613 [D loss: 0.264082, acc: 81.25%, op_acc: 57.81%] [G loss: 1.943908]
16614 [D loss: 0.310238, acc: 73.44%, op_acc: 53.12%] [G loss: 1.415157]
16615 [D loss: 0.182806, acc: 90.62%, op_acc: 60.94%] [G loss: 1.346373]
16616 [D loss: 0.213153, acc: 85.94%, op_acc: 59.38%] [G loss: 1.567533]
16617 [D loss: 0.251899, acc: 79.69%, op_acc: 57.81